# 01_slicing_texts

In [1]:
import numpy as np
import pandas as pd
import re
general_df = pd.read_csv('0_general_df.csv', sep=';', encoding='UTF-8')
general_df.head(2)

,Unnamed: 0,id,dokumentart,typ,vorgangsbezug_anzahl,dokumentnummer,wahlperiode,herausgeber,pdf_hash,datum,titel,text,fundstelle,vorgangsbezug,aktualisiert
0,0,5557,Plenarprotokoll,Dokument,55.0,20/91,20.0,BT,acf17f0dcd2b9245bc3b8e70ca673dd0,2023-03-16,Protokoll der 91. Sitzung des 20. Deutschen Bu...,[Deutscher Bundestag\nStenografischer Bericht\...,{'pdf_url': 'https://dserver.bundestag.de/btp/...,"[{'id': '285357', 'titel': 'Finanzierung des p...",2023-04-05T15:30:47+02:00
1,1,5556,Plenarprotokoll,Dokument,57.0,20/90,20.0,BT,5bc2491c29faa164a5255201c3b33496,2023-03-15,Protokoll der 90. Sitzung des 20. Deutschen Bu...,[Deutscher Bundestag\nStenografischer Bericht\...,{'pdf_url': 'https://dserver.bundestag.de/btp/...,"[{'id': '296088', 'titel': 'Neuntes Gesetz zur...",2023-03-17T09:38:07+01:00


In [2]:
general_df.columns

Index(['Unnamed: 0', 'id', 'dokumentart', 'typ', 'vorgangsbezug_anzahl',
       'dokumentnummer', 'wahlperiode', 'herausgeber', 'pdf_hash', 'datum',
       'titel', 'text', 'fundstelle', 'vorgangsbezug', 'aktualisiert'],
      dtype='object')

In [3]:
# Meu objetivo aqui é ver se tem alguma maneira de eu dividir os temas de uma sessao
# Vou dividir por Tagesordnungspunkte: do início até a primeira vez que foi chamado, entre o segundo e o terceiro, e etc.

# print(general_df['text'].iloc[2])  # Exemplo: 5 Tagesordnungpunkt

# Marcadores de início da sessao
#  Verificar primeira ocorrência
# Beginn: 9.00 Uhr

## Marcadores de Tagesordnung ?
# Ich rufe die Tagesordnungspunkte


# Marcadores de discurso:
# Vizepräsidentin Aydan Özoğuz:

# Franziska Mascheck (SPD):
# (CDU/CSU):

# print(general_df['text'].iloc[2])  # Exemplo: 5 Tagesordnungpunkt

In [4]:
#Criando o df que vai receber os TOP
colunas = ['id', 'dokumentart', 'typ', 'vorgangsbezug_anzahl', 'dokumentnummer', 'wahlperiode', 'herausgeber', 'pdf_hash', 'datum', 'titel', 'tagesordnungpunkt', 'fundstelle', 'vorgangsbezug', 'aktualisiert']
tagesordnungpunkt_df = pd.DataFrame(columns=colunas)

padrao = r"(Tagesordnungspunkt[e]?\s*.*?:)"


for index, row in general_df.iterrows():
    text = row['text']
    beginn_index = text.find('Beginn') # Encontra o índice da primeira ocorrência de "Beginn"
    resultado = text[beginn_index:]  # Extrai o texto a partir do índice encontrado
    trechos = re.split(padrao, resultado)

    # Criando df_temp:

    df_temp = pd.DataFrame(trechos, columns=["tagesordnungpunkt"])
    df_temp['id'] = row['id']
    df_temp['dokumentart'] = row['dokumentart']
    df_temp['typ'] = row['typ']
    df_temp['vorgangsbezug_anzahl'] = row['vorgangsbezug_anzahl']
    df_temp['dokumentnummer'] = row['dokumentnummer']
    df_temp['wahlperiode'] = row['wahlperiode']
    df_temp['herausgeber'] = row['herausgeber']
    df_temp['pdf_hash'] = row['pdf_hash']
    df_temp['datum'] = row['datum']
    df_temp['titel'] = row['titel']
    df_temp['fundstelle'] = row['fundstelle']
    df_temp['vorgangsbezug'] = row['vorgangsbezug']
    df_temp['aktualisiert'] = row['aktualisiert']

    tagesordnungpunkt_df = pd.concat([tagesordnungpunkt_df, df_temp])



In [5]:
tagesordnungpunkt_df.head(1)

,id,dokumentart,typ,vorgangsbezug_anzahl,dokumentnummer,wahlperiode,herausgeber,pdf_hash,datum,titel,tagesordnungpunkt,fundstelle,vorgangsbezug,aktualisiert
0,5557,Plenarprotokoll,Dokument,55.0,20/91,20.0,BT,acf17f0dcd2b9245bc3b8e70ca673dd0,2023-03-16,Protokoll der 91. Sitzung des 20. Deutschen Bu...,Beginn: 9.00 Uhr\nPräsidentin Bärbel Bas:\nSeh...,{'pdf_url': 'https://dserver.bundestag.de/btp/...,"[{'id': '285357', 'titel': 'Finanzierung des p...",2023-04-05T15:30:47+02:00


In [9]:
# Fazendo a verificacao:
# Criando a funcao:

def compare_arrays(arr1, arr2):
    """
    Compara dois ndarrays e imprime os valores divergentes, caso sejam diferentes.
    """

    # Verifica se os ndarrays são iguais
    if np.array_equal(arr1, arr2):
        print("Os ndarrays arr1 e arr2 são iguais.")
    else:
        # Encontra os índices dos valores divergentes
        divergent_indexes = np.where(arr1 != arr2)[0]

        print("Os ndarrays arr1 e arr2 são diferentes.")
        print("Valores divergentes:")
        for index in divergent_indexes:
            print(f"arr1[{index}] = {arr1[index]}, arr2[{index}] = {arr2[index]}")



# 0. Verificando todas as datas

arr1 = general_df['datum'].unique()
arr2 = tagesordnungpunkt_df['datum'].unique()
compare_arrays(arr1, arr2)
# As datas de um estao presentes no outro

# # 1. verificar se todas as sessoes estao aqui:

arr1 = general_df['dokumentnummer'].unique()
arr2 = tagesordnungpunkt_df['dokumentnummer'].unique()
compare_arrays(arr1, arr2)
# Os ndarrays são iguais. As sessoes que estao em um também estao em outro

# # 2. Verificar se tem um beginn: para cada sessao

top_beginn = tagesordnungpunkt_df[tagesordnungpunkt_df['tagesordnungpunkt'].str.startswith('Beginn:')]
arr3 = top_beginn['dokumentnummer'].unique()
compare_arrays(arr1, arr3)

# Valores divergentes:
# arr1[0] = 20/91, arr2[0] = 20/91
# Nao entendi

# # To impressionada que o Beginn: funcionou.

#Bom, aparentemente deu certo

Os ndarrays arr1 e arr2 são iguais.
Os ndarrays arr1 e arr2 são iguais.
Os ndarrays arr1 e arr2 são diferentes.
Valores divergentes:
arr1[0] = 20/91, arr2[0] = 20/91


C:\Users\JVoigt\AppData\Local\Temp\ipykernel_22840\3387450074.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  divergent_indexes = np.where(arr1 != arr2)[0]


In [ ]:
tagesordnungpunkt_df.to_csv("tagesordnungpunkt_df.csv", sep=';', encoding='UTF-8')